In [2]:
import os
import re
from urllib import request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm

In [3]:
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\sera\\hankyung_crawling'

In [5]:
#크롤링이 차단되어서 유저 에이전트(User Agent)를 지정
#headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'}
#headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36'}

In [6]:
pages = list(range(1,9797,1))

In [7]:
#urlretrieve 차단방지
opener = request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
request.install_opener(opener)

In [8]:
#크롤링이 차단되어서 유저 에이전트(User Agent)를 지정
from fake_useragent import UserAgent
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

user_agent = UserAgent()
headers = {'User-Agent': user_agent.random}
#response = requests.get(url, params=params, headers=headers)

## 2017.01.01~2022.04.04

In [9]:
data = pd.DataFrame()
pdf_error = pd.DataFrame()
page_error =[]

In [ ]:
for p in tqdm(pages):
    try:
        url ='http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate={}&edate={}&order_type=&now_page={}'.format(
    '2017-01-01','2022-04-04',p)

        html = requests.get(url,headers = headers)
        soup = BeautifulSoup(html.content, "html.parser")

        for i in range(1,len(soup.select('tr'))):
            date = soup.select('tr')[i].td.get_text('',strip=True)
            category=soup.select('tr')[i].find_all('td')[1].get_text('',strip=True)
            title = soup.select('tr')[i].li.get_text('',strip=True)
            author = soup.select('tr')[i].find_all('td')[-3].get_text('',strip=True)
            company = soup.select('tr')[i].find_all('td')[-2].get_text('',strip=True)
            tmp = pd.DataFrame(zip([date],[category],[title],[author],[company]))

            data = pd.concat([data,tmp],ignore_index=True)

            #download_pdf
            dlurl = 'http://consensus.hankyung.com'+soup.select('div.dv_input')[i-1].a.get('href')
            try : 
                request.urlretrieve(dlurl, './pdf_file/'+soup.select('div.dv_input')[i-1].a.get('title'))
            except : 
                tmp2 = pd.DataFrame(zip([p],[soup.select('div.dv_input')[i-1].a.get('title')]))
                pdf_error = pd.concat([pdf_error,tmp2],ignore_index=True)
                pass
    except : 
        page_error.append(p)
        pass
            


 54%|████████████████████████████████████████▎                                 | 5333/9796 [2:03:46<2:32:04,  2.04s/it]

In [ ]:
data.to_csv('hankyung_report.csv',index=False)

In [ ]:
page_error